In [4]:
import pandas as pd
import pickle
import nltk
nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords

from gensim.models.phrases import Phrases, Phraser

import itertools

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Денис\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Денис\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Source data

In [ ]:
geo = pd.read_excel('geo_comment.xlsx')
comment = geo['comment']

# Prepare text

First, we should lemmatize text, remove punctuation (leave only dots)

### Yandex Mystem

In [39]:
from pymystem3 import Mystem


def lemma(word):    
    m = Mystem()
    return m.lemmatize(word)[0]

### Better use Morphy

In [6]:
import pymorphy2


def lemma(word):
    morph = pymorphy2.MorphAnalyzer()
    p = morph.parse(word)[0]
    return p.normal_form

In [ ]:
%%time

# tokenize to words
# lemmatize them
# and then rebuild the text
def clean_text(text):
    # tokenize to words
    parts = nltk.word_tokenize(text)
    
    restrict_parts = [':', ',', ';']
    
    # allow punctuation
    punct = ['.', '-']
    
    result = ''
    
    for part in parts:
        if part not in restrict_parts:
            if part in punct:
                word = part
            # lemmatize
            else:
                word = ' ' + lemma(part) 
            result += word
    
    return result
    
    
#comment['comment'] = comment['comment'].apply(clean_text)
new_comment = []
i = 0
for c in comment:
    new_comment.append(clean_text(c))
    i += 1
    print(f'Done {i} of ', len(comment))
    
f = open('normalized_comments.pkl', 'rb')
pickle.dump(new_comment, f)
f.close()

Done 1 of  70382
Done 2 of  70382
Done 3 of  70382
Done 4 of  70382
Done 5 of  70382
Done 6 of  70382
Done 7 of  70382
Done 8 of  70382
Done 9 of  70382
Done 10 of  70382
Done 11 of  70382
Done 12 of  70382
Done 13 of  70382
Done 14 of  70382
Done 15 of  70382
Done 16 of  70382
Done 17 of  70382
Done 18 of  70382
Done 19 of  70382
Done 20 of  70382
Done 21 of  70382
Done 22 of  70382
Done 23 of  70382
Done 24 of  70382
Done 25 of  70382
Done 26 of  70382
Done 27 of  70382
Done 28 of  70382
Done 29 of  70382
Done 30 of  70382
Done 31 of  70382
Done 32 of  70382
Done 33 of  70382
Done 34 of  70382
Done 35 of  70382
Done 36 of  70382
Done 37 of  70382
Done 38 of  70382
Done 39 of  70382
Done 40 of  70382
Done 41 of  70382
Done 42 of  70382
Done 43 of  70382
Done 44 of  70382
Done 45 of  70382
Done 46 of  70382
Done 47 of  70382
Done 48 of  70382
Done 49 of  70382
Done 50 of  70382
Done 51 of  70382
Done 52 of  70382
Done 53 of  70382
Done 54 of  70382
Done 55 of  70382
Done 56 of  70382
D

Done 438 of  70382
Done 439 of  70382
Done 440 of  70382
Done 441 of  70382
Done 442 of  70382
Done 443 of  70382
Done 444 of  70382
Done 445 of  70382
Done 446 of  70382
Done 447 of  70382
Done 448 of  70382
Done 449 of  70382
Done 450 of  70382
Done 451 of  70382
Done 452 of  70382
Done 453 of  70382
Done 454 of  70382
Done 455 of  70382
Done 456 of  70382
Done 457 of  70382
Done 458 of  70382
Done 459 of  70382
Done 460 of  70382
Done 461 of  70382
Done 462 of  70382
Done 463 of  70382
Done 464 of  70382
Done 465 of  70382
Done 466 of  70382
Done 467 of  70382
Done 468 of  70382
Done 469 of  70382
Done 470 of  70382
Done 471 of  70382
Done 472 of  70382
Done 473 of  70382
Done 474 of  70382
Done 475 of  70382
Done 476 of  70382
Done 477 of  70382
Done 478 of  70382
Done 479 of  70382
Done 480 of  70382
Done 481 of  70382
Done 482 of  70382
Done 483 of  70382
Done 484 of  70382
Done 485 of  70382
Done 486 of  70382
Done 487 of  70382
Done 488 of  70382
Done 489 of  70382
Done 490 of 

Done 870 of  70382
Done 871 of  70382
Done 872 of  70382
Done 873 of  70382
Done 874 of  70382
Done 875 of  70382
Done 876 of  70382
Done 877 of  70382
Done 878 of  70382
Done 879 of  70382
Done 880 of  70382
Done 881 of  70382
Done 882 of  70382
Done 883 of  70382
Done 884 of  70382
Done 885 of  70382
Done 886 of  70382
Done 887 of  70382
Done 888 of  70382
Done 889 of  70382
Done 890 of  70382
Done 891 of  70382
Done 892 of  70382
Done 893 of  70382
Done 894 of  70382
Done 895 of  70382
Done 896 of  70382
Done 897 of  70382
Done 898 of  70382
Done 899 of  70382
Done 900 of  70382
Done 901 of  70382
Done 902 of  70382
Done 903 of  70382
Done 904 of  70382
Done 905 of  70382
Done 906 of  70382
Done 907 of  70382
Done 908 of  70382
Done 909 of  70382
Done 910 of  70382
Done 911 of  70382
Done 912 of  70382
Done 913 of  70382
Done 914 of  70382
Done 915 of  70382
Done 916 of  70382
Done 917 of  70382
Done 918 of  70382
Done 919 of  70382
Done 920 of  70382
Done 921 of  70382
Done 922 of 

Done 1287 of  70382
Done 1288 of  70382
Done 1289 of  70382
Done 1290 of  70382
Done 1291 of  70382
Done 1292 of  70382
Done 1293 of  70382
Done 1294 of  70382
Done 1295 of  70382
Done 1296 of  70382
Done 1297 of  70382
Done 1298 of  70382
Done 1299 of  70382
Done 1300 of  70382
Done 1301 of  70382
Done 1302 of  70382
Done 1303 of  70382
Done 1304 of  70382
Done 1305 of  70382
Done 1306 of  70382
Done 1307 of  70382
Done 1308 of  70382
Done 1309 of  70382
Done 1310 of  70382
Done 1311 of  70382
Done 1312 of  70382
Done 1313 of  70382
Done 1314 of  70382
Done 1315 of  70382
Done 1316 of  70382
Done 1317 of  70382
Done 1318 of  70382
Done 1319 of  70382
Done 1320 of  70382
Done 1321 of  70382
Done 1322 of  70382
Done 1323 of  70382
Done 1324 of  70382
Done 1325 of  70382
Done 1326 of  70382
Done 1327 of  70382
Done 1328 of  70382
Done 1329 of  70382
Done 1330 of  70382
Done 1331 of  70382
Done 1332 of  70382
Done 1333 of  70382
Done 1334 of  70382
Done 1335 of  70382
Done 1336 of  70382


Done 1697 of  70382
Done 1698 of  70382
Done 1699 of  70382
Done 1700 of  70382
Done 1701 of  70382
Done 1702 of  70382
Done 1703 of  70382
Done 1704 of  70382
Done 1705 of  70382
Done 1706 of  70382
Done 1707 of  70382
Done 1708 of  70382
Done 1709 of  70382
Done 1710 of  70382
Done 1711 of  70382
Done 1712 of  70382
Done 1713 of  70382
Done 1714 of  70382
Done 1715 of  70382
Done 1716 of  70382
Done 1717 of  70382
Done 1718 of  70382
Done 1719 of  70382
Done 1720 of  70382
Done 1721 of  70382
Done 1722 of  70382
Done 1723 of  70382
Done 1724 of  70382
Done 1725 of  70382
Done 1726 of  70382
Done 1727 of  70382
Done 1728 of  70382
Done 1729 of  70382
Done 1730 of  70382
Done 1731 of  70382
Done 1732 of  70382
Done 1733 of  70382
Done 1734 of  70382
Done 1735 of  70382
Done 1736 of  70382
Done 1737 of  70382
Done 1738 of  70382
Done 1739 of  70382
Done 1740 of  70382
Done 1741 of  70382
Done 1742 of  70382
Done 1743 of  70382
Done 1744 of  70382
Done 1745 of  70382
Done 1746 of  70382


Done 2107 of  70382
Done 2108 of  70382
Done 2109 of  70382
Done 2110 of  70382
Done 2111 of  70382
Done 2112 of  70382
Done 2113 of  70382
Done 2114 of  70382
Done 2115 of  70382
Done 2116 of  70382
Done 2117 of  70382
Done 2118 of  70382
Done 2119 of  70382
Done 2120 of  70382
Done 2121 of  70382
Done 2122 of  70382
Done 2123 of  70382
Done 2124 of  70382
Done 2125 of  70382
Done 2126 of  70382
Done 2127 of  70382
Done 2128 of  70382
Done 2129 of  70382
Done 2130 of  70382
Done 2131 of  70382
Done 2132 of  70382
Done 2133 of  70382
Done 2134 of  70382
Done 2135 of  70382
Done 2136 of  70382
Done 2137 of  70382
Done 2138 of  70382
Done 2139 of  70382
Done 2140 of  70382
Done 2141 of  70382
Done 2142 of  70382
Done 2143 of  70382
Done 2144 of  70382
Done 2145 of  70382
Done 2146 of  70382
Done 2147 of  70382
Done 2148 of  70382
Done 2149 of  70382
Done 2150 of  70382
Done 2151 of  70382
Done 2152 of  70382
Done 2153 of  70382
Done 2154 of  70382
Done 2155 of  70382
Done 2156 of  70382


Done 2517 of  70382
Done 2518 of  70382
Done 2519 of  70382
Done 2520 of  70382
Done 2521 of  70382
Done 2522 of  70382
Done 2523 of  70382
Done 2524 of  70382
Done 2525 of  70382
Done 2526 of  70382
Done 2527 of  70382
Done 2528 of  70382
Done 2529 of  70382
Done 2530 of  70382
Done 2531 of  70382
Done 2532 of  70382
Done 2533 of  70382
Done 2534 of  70382
Done 2535 of  70382
Done 2536 of  70382
Done 2537 of  70382
Done 2538 of  70382
Done 2539 of  70382
Done 2540 of  70382
Done 2541 of  70382
Done 2542 of  70382
Done 2543 of  70382
Done 2544 of  70382
Done 2545 of  70382
Done 2546 of  70382
Done 2547 of  70382
Done 2548 of  70382
Done 2549 of  70382
Done 2550 of  70382
Done 2551 of  70382
Done 2552 of  70382
Done 2553 of  70382
Done 2554 of  70382
Done 2555 of  70382
Done 2556 of  70382
Done 2557 of  70382
Done 2558 of  70382
Done 2559 of  70382
Done 2560 of  70382
Done 2561 of  70382
Done 2562 of  70382
Done 2563 of  70382
Done 2564 of  70382
Done 2565 of  70382
Done 2566 of  70382


Done 2927 of  70382
Done 2928 of  70382
Done 2929 of  70382
Done 2930 of  70382
Done 2931 of  70382
Done 2932 of  70382
Done 2933 of  70382
Done 2934 of  70382
Done 2935 of  70382
Done 2936 of  70382
Done 2937 of  70382
Done 2938 of  70382
Done 2939 of  70382
Done 2940 of  70382
Done 2941 of  70382
Done 2942 of  70382
Done 2943 of  70382
Done 2944 of  70382
Done 2945 of  70382
Done 2946 of  70382
Done 2947 of  70382
Done 2948 of  70382
Done 2949 of  70382
Done 2950 of  70382
Done 2951 of  70382
Done 2952 of  70382
Done 2953 of  70382
Done 2954 of  70382
Done 2955 of  70382
Done 2956 of  70382
Done 2957 of  70382
Done 2958 of  70382
Done 2959 of  70382
Done 2960 of  70382
Done 2961 of  70382
Done 2962 of  70382
Done 2963 of  70382
Done 2964 of  70382
Done 2965 of  70382
Done 2966 of  70382
Done 2967 of  70382
Done 2968 of  70382
Done 2969 of  70382
Done 2970 of  70382
Done 2971 of  70382
Done 2972 of  70382
Done 2973 of  70382
Done 2974 of  70382
Done 2975 of  70382
Done 2976 of  70382


Done 3337 of  70382
Done 3338 of  70382
Done 3339 of  70382
Done 3340 of  70382
Done 3341 of  70382
Done 3342 of  70382
Done 3343 of  70382
Done 3344 of  70382
Done 3345 of  70382
Done 3346 of  70382
Done 3347 of  70382
Done 3348 of  70382
Done 3349 of  70382
Done 3350 of  70382
Done 3351 of  70382
Done 3352 of  70382
Done 3353 of  70382
Done 3354 of  70382
Done 3355 of  70382
Done 3356 of  70382
Done 3357 of  70382
Done 3358 of  70382
Done 3359 of  70382
Done 3360 of  70382
Done 3361 of  70382
Done 3362 of  70382
Done 3363 of  70382
Done 3364 of  70382
Done 3365 of  70382
Done 3366 of  70382
Done 3367 of  70382
Done 3368 of  70382
Done 3369 of  70382
Done 3370 of  70382
Done 3371 of  70382
Done 3372 of  70382
Done 3373 of  70382
Done 3374 of  70382
Done 3375 of  70382
Done 3376 of  70382
Done 3377 of  70382
Done 3378 of  70382
Done 3379 of  70382
Done 3380 of  70382
Done 3381 of  70382
Done 3382 of  70382
Done 3383 of  70382
Done 3384 of  70382
Done 3385 of  70382
Done 3386 of  70382


# Chunking

In [ ]:
%%time

# load sentences tokenizer
sent_token = nltk.data.load('tokenizers/punkt/russian.pickle')

# each comment to sentences
sentences = comment.apply(sent_token.tokenize)

# concencate in one list
sentences = itertools.chain.from_iterable(sentences.tolist())

# tokenize to words as gensim accept sentences in list of words format
sent_list = []
for sent in sentences:
    sent_list += [nltk.word_tokenize(sent)]

## create model

In [ ]:
%%time

# create phrases model     
phrases = Phrases(sent_list, min_count=10, threshold=10)

# Export the trained model = use less RAM, faster processing. Model updates no longer possible.
bigram = Phraser(phrases)

# save model
bigram.save("bigram_model.pkl")

## apply model

In [ ]:
%%time

def chunks(text, model):
    result = ''
    
    sentences = sent_token.tokenize(text)
    for sent in sentences:
        for part in model[nltk.word_tokenize(sent)]:
            # rebuild text
            if part != '.':
                result += part
            else:
                result += ' ' + part
        

bigram = Phraser.load("bigram_model.pkl")

chunked_comments = [chunks(x, bigram) for x in new_comment]    

# Document-term matrix

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer, CountVectorizer

# we must remove stop words
sw = stopwords.words('russian')
with open('comment_stop_words.txt', encoding='utf-8') as f:
    our_stop_words = f.read().splitlines()
    f.close()

sw = sw + our_stop_words

# We will remove digits
word_pattern = r'[a-zA-Zа-яА-Я]+'

cv = CountVectorizer(stop_words=sw, token_pattern=word_pattern)
data_cv = cv.fit_transform(chunked_comments)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())

#del cv
#del data_cv

# Save document term matrix to file
Need a lot of memory for saving it as Dataframe. So we save it as sparse matrix

In [5]:
f = open('dtm.pkl', 'wb')
pickle.dump(data_cv, f)
f.close()

# save countvectorizer
f = open('cv.pkl', 'wb')
pickle.dump(cv, f)
f.close()

In [ ]:
print('Text count: ', data_dtm.shape[0], ' Word count: ', data_dtm.shape[1])

# Word stats

In [ ]:
%%time

words = {}

for word in data_dtm.columns:
    # first lemmatize colum
    w = lemma(word)
    
    if w in words:
        words[w] += data_dtm[word].sum()
    else:
        words[w] = data_dtm[word].sum() 

# save to DF, sort
w = pd.DataFrame.from_dict(words, orient='index')
w.sort_values(by=0,ascending=False, inplace=True)

In [ ]:
# save to csv
w.to_csv('word_stats.csv', encoding='utf-8')

f = open('words.pkl', 'wb')
pickle.dump(words, f)
f.close()

In [11]:
f = open('words.pkl', 'rb')
words = pickle.load(f)
f.close()

In [ ]:
from matplotlib import pyplot as plt
from wordcloud import WordCloud
plt.figure(figsize=(15,12))

wc = WordCloud(background_color="white", colormap="Dark2", max_font_size=150, random_state=42)

wc = wc.generate_from_frequencies(words)
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")

plt.show()

# Look at freq of words
First of all, i've added some words for our stop dictionary.

As we see, most the messages are related to land use and construction.

# Topic modeling
* LDA
* LSI (Latent Semantic Indexing)
* NMF Non-Negative Matrix Factorization 

## LDA
As it in video

In [ ]:
%%time
from gensim import matutils, models
import scipy.sparse

# we have to transpose
# convert from df to sparse matrix
sparse_counts = scipy.sparse.csr_matrix(data_dtm.transpose())

# sparse matrix to corpus
corpus = matutils.Sparse2Corpus(sparse_counts)

corpus

In [ ]:
f = open('sparse2corpus.pkl', 'wb')
pickle.dump(corpus, f)
f.close()

In [ ]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, passes=10)
lda.print_topics()